<a href="https://colab.research.google.com/github/Theparamvrsingh/Email-Spam-Project/blob/main/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [ ]:
# Upgrade necessary packages to avoid dependency conflicts
!pip install --upgrade httpx googletrans==4.0.0-rc1 streamlit scikit-learn openai langsmith

# Importing the required libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score
from googletrans import Translator
import pandas as pd
import streamlit as st

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langsmith to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langsmith to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You mig

Data Collection and Pre-Processing


In [ ]:
import pandas as pd
from google.colab import files

# Upload file
uploaded = files.upload()

# Load the dataset
for file_name in uploaded.keys():
    data = pd.read_csv(file_name)
    print(f"Loaded {file_name}")
    display(data.head())


Saving mail_data.csv to mail_data.csv
Loaded mail_data.csv


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
raw_mail_data = pd.read_csv('mail_data.csv')


In [ ]:
import os
print(os.getcwd())


/content


In [ ]:
print(raw_mail_data)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [ ]:
#Now i will replace the null values with a null string
mail_data = raw_mail_data.where((pd.notnull(raw_mail_data)),'')

In [ ]:
#Now printing the first 5 rows of the dataframe
mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
#Checking the number of rows and columns in the dataframe
mail_data.shape

(5572, 2)

Label Encoding

In [ ]:
#Labelling spam mails as 0 and ham mails as 1

mail_data.loc[mail_data['Category'] == 'spam', 'Category',] = 0
mail_data.loc[mail_data['Category'] == 'ham', 'Category'] = 1

spam:0     and      ham:1

In [ ]:
#Separating the data as texts and label
X = mail_data['Message']
Y = mail_data['Category']

In [ ]:
print(X)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object


In [ ]:
print(Y)

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object


Splitting the data into training data and test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=3)

In [ ]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)


(5572,)
(4457,)
(1115,)


Feature Extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# Transform the text data to feature vectors
feature_extraction = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True)

# Fit and transform training data, then transform test data
X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

# Encode Y_train and Y_test labels
label_encoder = LabelEncoder()
Y_train = label_encoder.fit_transform(Y_train)
Y_test = label_encoder.transform(Y_test)


In [ ]:
print(X_train)

3075                  Don know. I did't msg him recently.
1787    Do you know why god created gap between your f...
1614                         Thnx dude. u guys out 2nite?
4304                                      Yup i'm free...
3266    44 7732584351, Do you want a New Nokia 3510i c...
                              ...                        
789     5 Free Top Polyphonic Tones call 087018728737,...
968     What do u want when i come back?.a beautiful n...
1667    Guess who spent all last night phasing in and ...
3321    Eh sorry leh... I din c ur msg. Not sad alread...
1688    Free Top ringtone -sub to weekly ringtone-get ...
Name: Message, Length: 4457, dtype: object


In [ ]:
print(X_train_features)

  (0, 2329)	0.38783870336935383
  (0, 3811)	0.34780165336891333
  (0, 2224)	0.413103377943378
  (0, 4456)	0.4168658090846482
  (0, 5413)	0.6198254967574347
  (1, 3811)	0.17419952275504033
  (1, 3046)	0.2503712792613518
  (1, 1991)	0.33036995955537024
  (1, 2956)	0.33036995955537024
  (1, 2758)	0.3226407885943799
  (1, 1839)	0.2784903590561455
  (1, 918)	0.22871581159877646
  (1, 2746)	0.3398297002864083
  (1, 2957)	0.3398297002864083
  (1, 3325)	0.31610586766078863
  (1, 3185)	0.29694482957694585
  (1, 4080)	0.18880584110891163
  (2, 6601)	0.6056811524587518
  (2, 2404)	0.45287711070606745
  (2, 3156)	0.4107239318312698
  (2, 407)	0.509272536051008
  (3, 7414)	0.8100020912469564
  (3, 2870)	0.5864269879324768
  (4, 2870)	0.41872147309323743
  (4, 487)	0.2899118421746198
  :	:
  (4454, 2855)	0.47210665083641806
  (4454, 2246)	0.47210665083641806
  (4455, 4456)	0.24920025316220423
  (4455, 3922)	0.31287563163368587
  (4455, 6916)	0.19636985317119715
  (4455, 4715)	0.30714144758811196
  (

Training the model using Logistic Regression

In [ ]:
model1 = LogisticRegression()
model2 = RandomForestClassifier(n_estimators=100, random_state=3)
model3 = GradientBoostingClassifier(n_estimators=100, random_state=3)

Create an ensemble model using VotingClassifier

In [ ]:
ensemble_model = VotingClassifier(estimators=[
    ('lr', model1),
    ('rf', model2),
    ('gb', model3)
], voting='hard')


Train the ensemble model

In [ ]:
ensemble_model.fit(X_train_features, Y_train)


VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier(random_state=3)),
                             ('gb',
                              GradientBoostingClassifier(random_state=3))])

Evaluating the trained model

In [ ]:
# prediction on training data

train_predictions = ensemble_model.predict(X_train_features)
train_accuracy = accuracy_score(Y_train, train_predictions)
print('Training Accuracy:', train_accuracy)


Training Accuracy: 0.9789095804352703


In [ ]:
# Calculate accuracy on training data
train_predictions = ensemble_model.predict(X_train_features)
accuracy_on_training_data = accuracy_score(Y_train, train_predictions)
print('Accuracy on training data:', accuracy_on_training_data)

# Calculate accuracy on test data
test_predictions = ensemble_model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test, test_predictions)
print('Accuracy on test data:', accuracy_on_test_data)


Accuracy on training data: 0.9789095804352703
Accuracy on test data: 0.968609865470852


In [ ]:
print('Accuracy on training data : ', accuracy_on_training_data)

Accuracy on training data :  0.9789095804352703


In [ ]:
# prediction on test data

test_predictions = ensemble_model.predict(X_test_features)
test_accuracy = accuracy_score(Y_test, test_predictions)
print('Test Accuracy:', test_accuracy)


Test Accuracy: 0.968609865470852


In [ ]:
print('Accuracy on test data : ', accuracy_on_test_data)

Accuracy on test data :  0.968609865470852


Initialize Google Translator

In [ ]:
translator = Translator()

Function to handle translation and prediction

In [ ]:
def translate_and_predict(email_text):
    # Detect and translate to English if not already
    detection = translator.detect(email_text)
    if detection.lang != 'en':
        email_text = translator.translate(email_text, dest='en').text

    # Convert the email text to features
    features = feature_extraction.transform([email_text])
    prediction = ensemble_model.predict(features)[0]

    return 'Ham' if prediction == 1 else 'Spam'


In [ ]:
# Importing necessary modules
import pickle

# Assuming best_rf is your trained model (e.g., RandomForestClassifier)
with open('model.pkl', 'wb') as file:
    pickle.dump(ensemble_model, file)


In [ ]:
import joblib

# Save the TfidfVectorizer
joblib.dump(feature_extraction, 'vectorizer.pkl')

# Download the vectorizer file
from google.colab import files
files.download('vectorizer.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>